In [1]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [177]:
#start at 'pharmacies by state' page
base_URL = 'https://www.cvs.com/store-locator/cvs-pharmacy-locations'
base_html = requests.get(base_URL).text
base_soup = BeautifulSoup(base_html, 'html.parser')


In [189]:
#gather links to every state's page
stateLinks = []
states = base_soup.find('div', attrs={'class': 'states'})
for ul in states.find_all('ul'):
    liList = ul.find_all('li')
    for li in liList[:-1]:
        link = li.a.get('href')
        stateLinks.append('https://www.cvs.com' + str(link))

In [190]:
stateLinks[0]

'https://www.cvs.com/store-locator/cvs-pharmacy-locations/Alabama'

In [191]:
#loop through every state and collect the links to every town's direct page
links = []
for URL in stateLinks:
    html = requests.get(URL).text
    soup = BeautifulSoup(html, 'html.parser')
    stateClass = soup.find('div', attrs={'class': 'states'})
    for ul in stateClass.find_all('ul'):
        liList = ul.find_all('li')
        for li in liList:
            link = li.a.get('href')
            links.append('https://www.cvs.com' + str(link))


In [284]:
print('Number of Cities: ' + str(len(links)))


Number of Cities: 4172


In [266]:
#go to every town's direct page, collect data for each pharmacy in the town, and store data as dataframe
storeData = pd.DataFrame()
for row,link in enumerate(links):
    linkHTML = requests.get(link)
    newSoup = BeautifulSoup(linkHTML.text, 'html.parser')
    

    stores = newSoup.find_all('div',attrs={'class':'each-store'})
    for store in stores:
        address = ''
        phoneNum = ''
        storeNum = ''

        for storeAttr in store.find_all('p'):
            if storeAttr['class'] == ['store-address']:
                address = list(storeAttr.stripped_strings)[0]
            elif storeAttr['class'] == ['phone-number']:
                if len(list(storeAttr.stripped_strings)) == 2:
                    phoneNum,storeNum = list(storeAttr.stripped_strings)
                elif storeAttr.a != None:
                    storeNum = list(storeAttr.a.stripped_strings)[0]
                storeNum = storeNum.strip('# ')
            
        storeInfo = pd.Series([address,phoneNum,storeNum])
        storeInfo.name = row
        storeData = storeData.append(storeInfo)
        
    if row % 100 == 0 and row != 0:
        print('checkpoint: {} cities\' data collected'.format(row))
    
storeData.columns = ['address', 'phone_number','store_number']
storeData = storeData.reset_index(drop=True)


checkpoint: 100 cities' data collected
checkpoint: 200 cities' data collected
checkpoint: 300 cities' data collected
checkpoint: 400 cities' data collected
checkpoint: 500 cities' data collected
checkpoint: 600 cities' data collected
checkpoint: 700 cities' data collected
checkpoint: 800 cities' data collected
checkpoint: 900 cities' data collected
checkpoint: 1000 cities' data collected
checkpoint: 1100 cities' data collected
checkpoint: 1200 cities' data collected
checkpoint: 1300 cities' data collected
checkpoint: 1400 cities' data collected
checkpoint: 1500 cities' data collected
checkpoint: 1600 cities' data collected
checkpoint: 1700 cities' data collected
checkpoint: 1800 cities' data collected
checkpoint: 1900 cities' data collected
checkpoint: 2000 cities' data collected
checkpoint: 2100 cities' data collected
checkpoint: 2200 cities' data collected
checkpoint: 2300 cities' data collected
checkpoint: 2400 cities' data collected
checkpoint: 2500 cities' data collected
checkpoin

In [282]:
print('Number of Stores: ' + str(storeData.shape[0]))

Number of Stores: 9303


In [275]:
#storeData.to_excel('CVS_store_data.xlsx')